# Протоколы аутентификации с нулевым разглашением

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел
from Crypto.Util import number

# КГПСЧ
from Crypto.Random import random

# Понадобится в протоколе Шнорра
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Hash import SHA256

# Немного математики
import gmpy2

# Для генерации идентификаторов клиентов
import uuid

---

In [4]:
def randint(a, b) -> gmpy2.mpz:
    """
    Генерация случайного числа в отрезке [a, b] с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_a = int(a)
    native_b = int(b)
    
    return gmpy2.mpz(random.randint(native_a, native_b))


def getrandbits(n) -> gmpy2.mpz:
    """
    Генерация случайного числа длиной n бит с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_n = int(n)
    
    return gmpy2.mpz(random.getrandbits(native_n))

----

## Протокол Фиата-Шамира
![Fiat-Shamir Protocol](./images/Fiat-Shamir_2.png)

In [5]:
class FiatShamirCA(object):
    PRIME_BITS = 1024
    
    def __init__(self):
        """
        Инициализация доверенного центра.
        Генерируются два простых числа и их произведение.
        """
        
        self._db = {}
        
        self._p = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._q = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._modulus = gmpy2.mul(self._p, self._q)
        
        trace('[FiatShamirCA]', f'Modulus = {self._modulus}')
        
        
    def register_client(self, client_id: uuid.UUID, client_public_key: gmpy2.mpz) -> None:
        """
        Регистрация клиента. На вход принимает идентификатор клиента и его открытый ключ.
        Ничего не возвращает.
        Выбрасывает исключение, если клиент уже существует.
        """
        
        trace('[FiatShamirCA]', f'''Attempting to register client {client_id} 
        with public key {client_public_key}''')
        
        if client_id in self._db:
            trace('[FiatShamirCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Теперь сохраняю открытый ключ
        #
        
        self._db[client_id] = client_public_key
        trace('[FiatShamirCA]', 'Client registered successfully')
        
        
    def get_public_key(self, client_id: uuid.UUID) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента. Получает на вход идентификатор клиента.
        Возвращает открытый ключ.
        Выбрасывает исключение, если пользователь отсутствует.
        """
        
        return self._db[client_id]
       
        
    @property
    def modulus(self) -> gmpy2.mpz:
        """
        Возвращает модуль.
        """
        
        return self._modulus

In [6]:
class FiatShamirClient(object):
    def __init__(self, modulus: gmpy2.mpz, *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz] = None):
        """
        Инициализация клиента с выработкой ключевой пары и идентификатора.
        Последний параметр требуются для демонстрации неудачной аутентификации.
        """
        
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        self._r = None
        
        self._private_key = FiatShamirClient._generate_coprime(modulus)
        self._public_key = gmpy2.powmod(self._private_key, 2, modulus) if fake_data is None else fake_data[1]
        
        trace('[FiatShamirClient]', f'Client {self._id}')
        trace('[FiatShamirClient]', f'Public key = {self._public_key}')
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора.
        """
        
        return self._id
    
    
    def get_x(self, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение случайного значения, вычисляемого по формуле:
                    x = r ** 2 mod n,
        где n - модуль, r - случайное целое число из отрезка [1, n - 1].
        Значение r сохраняется.
        """
        
        self._r = randint(1, modulus - 1)
        return gmpy2.powmod(self._r, 2, modulus)
    
    
    def get_y(self, e: gmpy2.mpz, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение числа, вычисляемого по формуле:
                    y = r * s ** e mod n,
        где n - модуль, s - закрытый ключ пользователя, r - сгенерированное 
        функцией get_x внутреннее значение, e - целое число из отрезка [0, 1],
        пришедшее от проверяющего.
        """
        
        product = gmpy2.mul(self._r, gmpy2.powmod(self._private_key, e, modulus))
        return gmpy2.t_mod(product, modulus)
        
        
    @staticmethod
    def _generate_coprime(modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Генерация случайного числа из отрезка [1, n - 1], взаимно
        простого с n, где n - модуль.
        """
        
        #
        # Генерирую число до тех пор, пока оно не станет взаимно простым с модулем
        #
        
        result = randint(1, modulus - 1)
        while gmpy2.gcd(result, modulus) != 1:
            result = randint(1, modulus - 1)
            
        return result

In [7]:
class FiatShamirVerifier(object):
    def __init__(self, iterations: int):
        """
        Инициализация проверяющего. Задает коичество итераций проверки.
        """
        
        self._iterations = iterations
        
        
    def authenticate(self, client: FiatShamirClient, ca: FiatShamirCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        trace('[FiatShamirVerifier]', f'Attempt to authenticate client {client.identifier}')
        trace('[FiatShamirVerifier]', f'Number of iterations = {self._iterations}')
        
        is_successful = True
        
        for i in range(self._iterations):
            x = client.get_x(ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {x = }')
            
            e = randint(0, 1)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {e = }')
            
            y = client.get_y(e, ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {y = }')
            
            pk = ca.get_public_key(client.identifier)
            product = gmpy2.mul(x, gmpy2.powmod(pk, e, ca.modulus))
            
            successful_iteration = gmpy2.powmod(y, 2, ca.modulus) == gmpy2.t_mod(product, ca.modulus)
            is_successful &= successful_iteration
            
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] success: {successful_iteration}')
            
            if not successful_iteration:
                break
            
        print(f'Authentication successful: {is_successful}')
        return is_successful

----

In [8]:
# Создаю доверенный центр
ca = FiatShamirCA()

[TRACE] [FiatShamirCA] Modulus = 13278809920564849048192301590773477588906409326272330307226733334831540274520906404803678702725156012439938169935138453281957126507511244238259023667338663089415181327105242318311371983894261231376179540214090059741865422158272421225941373676302171328693991602850418336071383579844628847211264894587851642450356591576978295589725993095190589519706150153791231897507488102742214336972545169130550607797108893280632409464066632113511979736996750307455425480493893647810545827437385739582399919564641748435795606686464078697496751761956660061216492080969289979847863759518339581310489962154382130112427727141999446340213


In [9]:
# Создаю двух пользователей:
# - Алису - валидного пользователя
# - Еву - пользователя, представляющегося Алисой. но не знающего закрытого ключа Алисы
alice = FiatShamirClient(ca.modulus)
eve = FiatShamirClient(ca.modulus, 
                       fake_data=(alice.identifier, alice.public_key))

[TRACE] [FiatShamirClient] Client f22b36b8-b9ed-476f-8654-2fbef5ae15e0
[TRACE] [FiatShamirClient] Public key = 9983682727313212491528435064815239236399168939138470093115952937522519056663298781367412581421900809415906719022875329067162232756560549540073709157759344310690253211223927509371448820387707894422876876630194342975259049095497687374196738637798937948652002100230452076492304147636383193275376680929961899282256411688067448319480366261089911921359781364490139086509278018597870482505827058054715131643911586224548717254169121621600986675873686166474518049771319753917801474548318038132174121979341483575495081605124003836127169796763478195015452008152019401865378682842512929396900425502017254808080602623038332046403
[TRACE] [FiatShamirClient] Client f22b36b8-b9ed-476f-8654-2fbef5ae15e0
[TRACE] [FiatShamirClient] Public key = 99836827273132124915284350648152392363991689391384700931159529375225190566632987813674125814219008094159067190228753290671622327565605495400737091577593443106902

In [10]:
# Регистрирую валидного пользователя
ca.register_client(alice.identifier, alice.public_key)

[TRACE] [FiatShamirCA] Attempting to register client f22b36b8-b9ed-476f-8654-2fbef5ae15e0 
        with public key 9983682727313212491528435064815239236399168939138470093115952937522519056663298781367412581421900809415906719022875329067162232756560549540073709157759344310690253211223927509371448820387707894422876876630194342975259049095497687374196738637798937948652002100230452076492304147636383193275376680929961899282256411688067448319480366261089911921359781364490139086509278018597870482505827058054715131643911586224548717254169121621600986675873686166474518049771319753917801474548318038132174121979341483575495081605124003836127169796763478195015452008152019401865378682842512929396900425502017254808080602623038332046403
[TRACE] [FiatShamirCA] Client registered successfully


In [11]:
# Для простоты будет 3 итерации
ITERATIONS = 3

# Создаю проверяющую сторону
verifier = FiatShamirVerifier(ITERATIONS)

In [12]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client f22b36b8-b9ed-476f-8654-2fbef5ae15e0
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(6257121833348624605957790855757805620010101467056323565053639234154521639671897488802474879220714453411210725519528149083408175575438079405860551073597194531213490326107943414924002033728216785437234917237005756786856149077898830648245492976532225710885155660594906120565402461020446034952848666884871028487458827524423438925821385723696112562938701443377687945736014135714960677146759209629380846353071415967370807015250134429322713945115878585900485100073534726232527315550557724154893465969473969085854319152735392573010439741070568441828185660265986305197807173343560667110489250495416404629188593882653415763479)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(876384330790188601483984831917905161527312126526795977047649556445781917185

True

In [13]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client f22b36b8-b9ed-476f-8654-2fbef5ae15e0
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(9757318661082508585898975527461302682314816649943497333157136606640354119090222770100791571522046842302570142069149827875503362383053904411393333497218353496682482111421898753169384845506819497682785495045495404804909698917327999169970935300302676338623625511255655854136526239638741533681245148169135307073262114647056165124228318485800814237572189235493181787047721033490032744994449602975750111068605649923455375602299895706604772755752772226152673011337243594587710822661608129525166730433873011859763286303811900280461071544731515507499829429674278418928784057428965867036775536137889032562938554351589669240544)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(651372337960456024359015951421744828776264123252835135263367482222928052204

False

---

## Протокол Шнорра
![Schnorr Protocol](./images/Schnorr_2.png)

In [14]:
class SchnorrCA(object):
    RSA_KEY_LENGTH = 4096
    
    def __init__(self):
        """
        Инициализация доверенного центра
        Генерация простых чисел p, q и элемента g, а также ключевой пары СА
        """
        
        self._db = {}
        
        #
        # Генерация параметров протокола
        #
        
        self._p, self._q, self._g = SchnorrCA._generate_parameters()
        
        #
        # Ключевая пара RSA для СА
        #
        
        self._private_key = RSA.generate(SchnorrCA.RSA_KEY_LENGTH)
        self._public_key = self._private_key.publickey()
        
        trace('[SchnorrCA]', f'RSA public key = {(self._public_key.e, self._public_key.n)}')
        
        
    def register_client(self, 
                        client_id: uuid.UUID, 
                        client_public_key: gmpy2.mpz) -> tuple[uuid.UUID, gmpy2.mpz, bytes]:
        """
        Регистрация клиента. Принимает на вход идентификатор клиента и его открытый ключ.
        Возвращает подписанный сервером сертификат.
        Выбрасывает исключение, если клиент уже зарегистрирован.
        """
        
        if client_id in self._db:
            trace('[SchnorrCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Создаю сертификат
        #
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        signature = pss.new(self._private_key).sign(h)
        
        certificate = (client_id, client_public_key, signature)
        trace('[SchnorrCA]', f'Certificate = {certificate}')
        
        #
        # Сохраняю его и возвращаю
        #
        
        self._db[client_id] = certificate
        return certificate
    
    
    def verify_certificate(self, certificate: tuple[uuid.UUID, gmpy2.mpz, bytes]) -> bool:
        """
        Проверка сертификата.
        Возвращает признак валидности сертификата.
        """
        
        client_id, client_public_key, signature = certificate
        
        if client_id not in self._db:
            trace('[SchnorrCA]', f"Client {client_id} doesn't exist")
            raise ValueError("Client doesn't exist")
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        verifier = pss.new(self.public_key)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа СА.
        """
        
        return self._public_key
    
    
    @property
    def parameters(self) -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Параметры протокола (p, q, g)
        """
        
        return self._p, self._q, self._g
        
        
    @staticmethod
    def _generate_parameters() -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Генерация параметров протокола (p, q, g).
        """
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = gmpy2.mpz(number.getPrime(Q_LENGTH))
        p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
        
        while not gmpy2.is_prime(p):
            p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
            
        #
        # Сгенерирую число, взаимно простое с p
        #
        
        tmp = getrandbits(P_LENGTH)
        while gmpy2.gcd(tmp, p) != 1:
            tmp = getrandbits(P_LENGTH)
        
        #
        # По малой теореме Ферма такой g будет иметь порядок q
        #
        
        g = gmpy2.powmod(tmp, (p - 1) // q, p)
        
        trace('[SchnorrCA]', f'{p = }, {q = }, {g = }')
        return p, q, g

In [15]:
class SchnorrClient(object):
    def __init__(self, parameters: tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz], *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz, tuple] = None):
        """
        Инициализация клиента. Генерирует идентификатор клиента,
        ключевую пару.
        Последний параметр необходим для демонстрации неуспешной аутентификации.
        """
        
        self._p, self._q, self._g = parameters
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        
        self._cert = None if fake_data is None else fake_data[2]
        self._r = None
        
        self._private_key = randint(1, self._q - 1)
        self._public_key  = gmpy2.powmod(
            self._g, -self._private_key, self._p) if fake_data is None else fake_data[1]
        
        trace('[SchnorrClient]', f'Client {self._id}')
        trace('[SchnorrClient]', f'Public key = {self._public_key}')
        
        
    def register(self, ca: SchnorrCA) -> None:
        """
        Регистрация клиента на СА. Принимает дескриптор СА.
        """
        
        self._cert = ca.register_client(self.identifier, self.public_key)
        
        
    def get_x(self) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = g ** r mod p
        где r - случайное целое число из отрезка [1, q - 1].
        Сохраняет значение r.
        """
        
        self._r = randint(1, self._q - 1)
        return gmpy2.powmod(self._g, self._r, self._p)
    
    
    def get_y(self, e, security_parameter) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = a * e + r mod q
        где r - сохраненное в get_x служебное значение, e - число,
        полученное от проверяющего, a - закрытый ключ клиента.
        """
        
        if e > 2 ** security_parameter or e < 1:
            trace('[SchnorrClient]', f'Invalid {e = }')
            raise ValueError('Invalid e')
            
        pre_result = gmpy2.mul(self._private_key, e) + self._r
        return gmpy2.t_mod(pre_result, self._q)
    
    
    @property
    def certificate(self):
        """
        Получение сертификата клиента.
        """
        
        return self._cert
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента.
        """
        
        return self._public_key
        
        
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id

In [16]:
class SchnorrVerifier(object):
    def __init__(self, parameters, security_parameter):
        """
        Инициализация проверяющего. Фактически только закрепляет параметры протокола.
        """
        
        self._p, self._q, self._g = parameters
        self._security_parameter = security_parameter
        
        assert security_parameter > 0 and 2 ** security_parameter < self._q
        
        self._e_values = set()
        
        
    def authenticate(self, client: SchnorrClient, ca: SchnorrCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        #
        # Получаю с клиента x и его сертификат
        #
        
        x, cert = client.get_x(), client.certificate
        trace('[SchnorrVerifier]', f'{x = }')
        
        #
        # Если сертификат битый или невалидный, то это плохо
        #
        
        if not ca.verify_certificate(cert):
            trace('[SchnorrVerifier]', f'Invalid certificate {cert}')
            return False
            
        #
        # Генерирую число е и отдаю его клиенту
        #
            
        e = randint(1, 2 ** self._security_parameter)
        while e in self._e_values:
            e = randint(1, 2 ** self._security_parameter)
            
        self._e_values.add(e)        
        trace('[SchnorrVerifier]', f'{e = }')
        
        y = client.get_y(e, self._security_parameter)
        trace('[SchnorrVerifier]', f'{y = }')
        
        #
        # Вычисляю значение, которое необходимо сравнить с полученным от клиента х
        # И сразу же сравниваю их
        # 
        
        product = gmpy2.mul(gmpy2.powmod(self._g, y, self._p), 
                            gmpy2.powmod(client.public_key, e, self._p))
        z = gmpy2.t_mod(product, self._p)
        trace('[SchnorrVerifier]', f'{z = }')
        
        is_successful = x == z
        print(f'Authentication successful: {is_successful}')
        
        return is_successful

----

In [17]:
# Создаю доверенный центр
ca = SchnorrCA()

[TRACE] [SchnorrCA] p = mpz(62106400067600944212588247453327570529541940886435123866558336218241283937792579585957908711005590595780309673655065468857957696755388551964934957117039662457181932784088870182460670688774716945137893868364281553716521452057932431952460432950711303256664425523976979290257460561771870581482268064803021103141), q = mpz(929442869135085249781999474488063596734285770721), g = mpz(33405455632790003158588259164297921868122998677747352122004448019887760966643587491122577445593062018305253032231377332482178201154833272069711554067257670610047436052196706697698377050998356965179265272077740325293487988434696215785334363331064417035502070571262085870484942359440881088680315823210354250671)
[TRACE] [SchnorrCA] RSA public key = (65537, 73000226284161273051679229921622998128638816502655374951085726769382759745588846738077424489870119378930908297622610387608955752338148547901361197294870830656213527607823849645466131278408527599295174695889373645569499321099870304990720192

In [18]:
# Создам Алису - валидного пользователя
alice = SchnorrClient(ca.parameters)

[TRACE] [SchnorrClient] Client 8818538e-5cca-47f3-a081-b5aa035047a4
[TRACE] [SchnorrClient] Public key = 4461329321959763524015883479792024188971128175650713139523025478580013601461563643411626552883364787966095077529685074701698333316947849860550358746972792481697705941632252198438490872843360926877876947060213063207720658400381176333289966228334783945159657256708166927754303108155336664237509729364769871329


In [19]:
# Регистрирую валидного пользователя
alice.register(ca)

[TRACE] [SchnorrCA] Certificate = (UUID('8818538e-5cca-47f3-a081-b5aa035047a4'), mpz(4461329321959763524015883479792024188971128175650713139523025478580013601461563643411626552883364787966095077529685074701698333316947849860550358746972792481697705941632252198438490872843360926877876947060213063207720658400381176333289966228334783945159657256708166927754303108155336664237509729364769871329), b'\t-lk\xa3e\xa2\xc5\xf3\'\xef\xeb\x8c\xa72K\x1f\n\xd3\xdc\x0bH,_K-\xbb\xaai\xd8\xec\x98\xde:\x06T\xc8r2\x16\x0e\xb7\xf5\xa0\xf8@\xfa*E\xee\xee\x0c\xdc@\x829\x9f\x9a\xc3o\x07\xa8^T\xdf\xfd\x8ch\xfb\xccT\x11\\\xc2\x06\x1d\x86\xd5\xa0\x17e\x08P\xb2K\x96\xeb\x7f\x0cQ\x98|\xe3d\xe16\x11\x89\xd7\x80)#\\\x9c\x930\xdc\x83\xe0qN\xcf\x9e\x10)\xd9>L\xf8f\xd6\x8e?\n8\x86\xe1S\xb1\xb6\x15\t\xd3\xd6\x16\xe2\xe2\xeb11\x06\xe4\xa7\x01\x0c\xf1\xd6\x17u\x06\xde/I\t(MM\x94\xd15\xfb4\xf3\xb7\x84R9\xf4\xf0\t\xef\xc3]\xbbOJ\xb6\xa6\x92\xd8\xdd\xfe\x80\xb1\xb0\x9d\x9d\x93\xb2\x1b\x1b\x07\x02\x15QO\x86F\x9d\xfc+\x10\x9dP

In [20]:
# Параметр безопасности положим равным 10 для демонстрации
security_parameter = 10

# Создаю проверяющую сторону
verifier = SchnorrVerifier(ca.parameters, security_parameter)

In [21]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [SchnorrVerifier] x = mpz(8697851823431680926300591576611941726999333842794196781775916647406471483953498592645870306848042127142165885855560548427377523582309857909597513674104002492363656797628619145407963963456928217209590568762023666065695666454058685089351000982129824601842697005942334965127574854735101804511841403155121990749)
[TRACE] [SchnorrVerifier] e = mpz(781)
[TRACE] [SchnorrVerifier] y = mpz(228509402956058093924887459921964500406684761572)
[TRACE] [SchnorrVerifier] z = mpz(8697851823431680926300591576611941726999333842794196781775916647406471483953498592645870306848042127142165885855560548427377523582309857909597513674104002492363656797628619145407963963456928217209590568762023666065695666454058685089351000982129824601842697005942334965127574854735101804511841403155121990749)
Authentication successful: True


True

In [22]:
# Создаю Еву - клиента, который пытается представиться Алисой
eve = SchnorrClient(ca.parameters, 
                    fake_data=(alice.identifier, alice.public_key, alice.certificate))

[TRACE] [SchnorrClient] Client 8818538e-5cca-47f3-a081-b5aa035047a4
[TRACE] [SchnorrClient] Public key = 4461329321959763524015883479792024188971128175650713139523025478580013601461563643411626552883364787966095077529685074701698333316947849860550358746972792481697705941632252198438490872843360926877876947060213063207720658400381176333289966228334783945159657256708166927754303108155336664237509729364769871329


In [23]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [SchnorrVerifier] x = mpz(50474581909447865464909018568412075097791513270736029353848782266310950460750940422150894021616212038150709845170805735086305162942045799746730700430003153967507800530311649364166949758278045192818014759820961788445473512332730883679503295781082261519294215233489695174646708133705771464652135746108954384803)
[TRACE] [SchnorrVerifier] e = mpz(616)
[TRACE] [SchnorrVerifier] y = mpz(12442844262399920884636175024587406756838055163)
[TRACE] [SchnorrVerifier] z = mpz(28386474297241265803066621177828913006359591792985780468488234066111322436007807716373559352881179649070010323157378993613305667082155001423738562005453007826218427044224601356536148495079299089932153172277585582632502726111007459371813700739648232301358922226954327746443346657648639981789461555664167364363)
Authentication successful: False


False